In [3]:
import sys
import getpass
sys.path.append('..')

In [4]:
import pandas as pd
import os
import re
from sqlalchemy import create_engine, text, inspect, MetaData, Table, Column, Integer, Float, String, Text
from sqlalchemy.orm import sessionmaker
from geoalchemy2 import Geography, WKTElement
from FlowHandler import FlowHandler
from Akvo import Flow

In [5]:
PSQL_USER = getpass.getuser()
PSQL_PWD = os.environ['KEYCLOAK_PWD']
PSQL_DB = 'flow_test'

In [6]:
instanceURI = 'angkorsalad'
requestURI = 'https://api.akvo.org/flow/orgs/' + instanceURI
surveysUrl = []

In [7]:
def getFolders(items):
    for folder in items['folders']:
        try:
            surveysUrl.append(folder['surveysUrl'])
            childs = Flow.getResponse(folder['foldersUrl'])
            getFolders(childs)
        except:
            pass

In [8]:
parents = Flow.getResponse(requestURI + '/folders')
getFolders(parents)

In [9]:
surveys = []
for surveyUrl in surveysUrl:
    surveyList = Flow.getResponse(surveyUrl)
    for survey in surveyList['surveys']:
        surveys.append(survey)

In [16]:
survey_example = surveys[1]

In [17]:
Flow.getResponse(survey_example['surveyUrl'])

{'id': '17500002',
 'name': 'Yield Data Collection',
 'registrationFormId': '',
 'forms': [{'id': '5330003',
   'name': 'Yield Data Collection',
   'questionGroups': [{'id': '17490004',
     'name': 'Basic Informations',
     'repeatable': False,
     'questions': [{'id': '27610004',
       'name': 'Soil Sample Code',
       'type': 'FREE_TEXT',
       'order': 1,
       'variableName': None,
       'createdAt': '2019-01-30T05:47:14.145Z',
       'modifiedAt': '2019-02-20T20:23:21.949Z'},
      {'id': '37550003',
       'name': 'Crop Name',
       'type': 'CASCADE',
       'order': 2,
       'variableName': None,
       'createdAt': '2019-01-30T05:50:14.844Z',
       'modifiedAt': '2019-01-30T05:50:39.131Z'},
      {'id': '19470001',
       'name': 'Technician Name',
       'type': 'FREE_TEXT',
       'order': 3,
       'variableName': None,
       'createdAt': '2019-01-30T05:47:47.848Z',
       'modifiedAt': '2019-01-30T05:50:39.131Z'},
      {'id': '17490005',
       'name': 'Name of